#                                     Identifying Coding Languages:

### Project Plan:

##### Acquire:
    - To Acquire the data we had to grab a github access token so that we could scrape their website using our new credentials.


    - We then grabbed the readme files out of 100 pages of repositories that were 'space' and were either using Python, Java, Javascript, or C#.


##### Prepare:
    - The readme's we acquired are filled with links and metadata text which is not going to be useful for our models so in order to clean it we had to:
        -Get rid of links


##### Explore:

##### Model:

##### Deliver